In [1]:
import numpy as np
import gpxpy
import matplotlib.pyplot as plt
import pandas as pd
import folium
import geopy.distance
import math
import branca

In [2]:
# PARAMETERS #
# Path to GPX file
gpx = gpxpy.parse(open('test_track.gpx','r'))

# vehicle specifications
topspeed = 180 # mph
cornering = math.tan(math.radians(45)) # g

# CONSTANTS #
dx = 2 # curve smoothing, 1 for none, 2+ for smoother curves but less accuracy in tighter corners
corner_sensitivity = 6 # multiple of top speed to mark corners, higher for less straights marked
shortest_straight = 0.1 # miles, shortest straight to mark
min_angle_change = 5 # degrees, minimum angle change

In [3]:
# standardize units in specs
topspeed = topspeed / 2.237 # convert to m/s and add 1 m/s for safety margin
cornering = cornering * 9.81 # convert to m/s^2
shortest_straight = shortest_straight / 0.000621371 # convert to m


# Fix descriptions in GPX file
for p in gpx.tracks[0].segments[0].points:
    if p.description != None:
        
        # split description into list based on <
        desc = p.description.split('<')
        #print(desc)
        # if string in list contains >, split on > and take second element
        for i in range(len(desc)):
            if '>' in desc[i]:
                desc[i] = desc[i].split('>')[1]

        # merge list into string
        desc = ' '.join(desc)

        # write new description
        p.description = desc
    else:
        p.name = None

# remove duplicate points
old = (0,0)
for p in gpx.tracks[0].segments[0].points:
    if (p.latitude, p.longitude) == old:
        gpx.tracks[0].segments[0].points.remove(p)

    old = (p.latitude, p.longitude)


# calculate curve radius of each point in GPX file and add to point, along with curve direction and speed
radii = []; direction = []; speed = []
max_radius = ((topspeed)**2)/(cornering)
for p in gpx.tracks[0].segments[0].points:
    
    # ignore beginning and end of track
    if gpx.tracks[0].segments[0].points.index(p) < dx or gpx.tracks[0].segments[0].points.index(p) >= len(gpx.tracks[0].segments[0].points) - dx:
        radii.append(max_radius)
        direction.append(0)
    
    else:
        # find previous and next points
        prev_point = gpx.tracks[0].segments[0].points[gpx.tracks[0].segments[0].points.index(p)-dx]
        next_point = gpx.tracks[0].segments[0].points[gpx.tracks[0].segments[0].points.index(p)+dx]

        # calculate distance between points
        a = ((prev_point.latitude - p.latitude)**2 + (prev_point.longitude - p.longitude)**2)**0.5
        b = ((next_point.latitude - p.latitude)**2 + (next_point.longitude - p.longitude)**2)**0.5
        c = ((next_point.latitude - prev_point.latitude)**2 + (next_point.longitude - prev_point.longitude)**2)**0.5

        if a*b*c > 0 and a+b-c>0:
            cosangle = (a**2 + b**2 - c**2) /(2*a*b)
            try:
                sinangle = math.sqrt(1 - cosangle**2)
                radius = c / sinangle
            except: pass
        scale = geopy.distance.distance((p.latitude, p.longitude), (next_point.latitude, next_point.longitude)).m / b
        radius *= scale
        
        radii.append(radius)

        # calculate direction of curve
        if (next_point.latitude - p.latitude)*(prev_point.longitude - p.longitude) - (next_point.longitude - p.longitude)*(prev_point.latitude - p.latitude) > 0 and radius < max_radius*corner_sensitivity:
            direction.append(1)
        elif (next_point.latitude - p.latitude)*(prev_point.longitude - p.longitude) - (next_point.longitude - p.longitude)*(prev_point.latitude - p.latitude) < 0 and radius < max_radius*corner_sensitivity:
            direction.append(2)
        else:
            direction.append(0)

    # calculate speed
    if radii[-1] >= max_radius:
        speed.append(topspeed)
    else:
        speed.append(cornering * (radii[-1])**0.5 / 2)



# split track into segments based on direction
segments = [[0]]
for p in gpx.tracks[0].segments[0].points:
    if direction[gpx.tracks[0].segments[0].points.index(p)] != direction[segments[-1][-1]]:
        segments.append([gpx.tracks[0].segments[0].points.index(p)])
    else:
        segments[-1].append(gpx.tracks[0].segments[0].points.index(p))

# determine if segments are straights or curves, repackage
for s in segments:
    if direction[s[0]] == 0:
        sdir = 'straight'
    elif direction[s[0]] == 1:
        sdir = 'right'
    elif direction[s[0]] == 2:
        sdir = 'left'
    segments[segments.index(s)] = [sdir, s]


# clean up segments
for i in range(5):
    # if segment is too short, combine with neighboring segment
    for s in segments:
        if len(s[1]) == 1 and segments.index(s) != 0 and segments.index(s) != len(segments)-1:
            if segments[segments.index(s)-1] > segments[segments.index(s)+1]:
                segments[segments.index(s)-1][1] += s[1]
                segments.remove(s)
            else:
                segments[segments.index(s)+1][1] = s[1] + segments[segments.index(s)+1][1]
                segments.remove(s)

    # if neighboring segments are both same type, combine
    for s in segments:
        if segments.index(s) != 0:
            if s[0] == segments[segments.index(s)-1][0]:
                segments[segments.index(s)-1][1] += s[1]
                segments.remove(s)

# determine angle change for each segment
for s in segments:
    if s[0] == 'straight':
        segments[segments.index(s)].append(0)
    else:
        for p in s[1]:
            if gpx.tracks[0].segments[0].points.index(gpx.tracks[0].segments[0].points[p]) == s[1][0]:
                start = gpx.tracks[0].segments[0].points[p]
            elif gpx.tracks[0].segments[0].points.index(gpx.tracks[0].segments[0].points[p]) == s[1][-1]:
                end = gpx.tracks[0].segments[0].points[p]

        # calculate angle change based on start and end directions
        # end cases fixed by considering points in opposite direction
        
        if s[0] == 'right':
            start_angle = math.atan2(start.latitude - gpx.tracks[0].segments[0].points[s[1][0]-1].latitude, start.longitude - gpx.tracks[0].segments[0].points[s[1][0]-1].longitude)
            end_angle = math.atan2(end.latitude - gpx.tracks[0].segments[0].points[s[1][-1]+1].latitude, end.longitude - gpx.tracks[0].segments[0].points[s[1][-1]+1].longitude)
        elif s[0] == 'left':
            start_angle = math.atan2(gpx.tracks[0].segments[0].points[s[1][0]-1].latitude - start.latitude, gpx.tracks[0].segments[0].points[s[1][0]-1].longitude - start.longitude)
            end_angle = math.atan2(gpx.tracks[0].segments[0].points[s[1][-1]+1].latitude - end.latitude, gpx.tracks[0].segments[0].points[s[1][-1]+1].longitude - end.longitude)
        
        angle_change = end_angle - start_angle
        if angle_change < 0:
            angle_change += 2*math.pi
        angle_change = math.degrees(angle_change) - 180

        segments[segments.index(s)].append(np.abs(angle_change))

# calculate distance for each segment
for s in segments:
    s.append(geopy.distance.distance((gpx.tracks[0].segments[0].points[s[1][0]].latitude, gpx.tracks[0].segments[0].points[s[1][0]].longitude), (gpx.tracks[0].segments[0].points[s[1][-1]].latitude, gpx.tracks[0].segments[0].points[s[1][-1]].longitude)).m)

# split 2-length segments between neighboring segments
for s in segments:
    if len(s[1]) == 2:
        if segments.index(s) != 0 and segments.index(s) != len(segments)-1:
            segments[segments.index(s)-1][1] += [s[1][0]]
            segments[segments.index(s)+1][1] = [s[1][1]] + segments[segments.index(s)+1][1]
            segments.remove(s)


# clean up segments
for i in range(5):
    # if segment is too short, combine with neighboring segment
    for s in segments:
        if len(s[1]) == 1 and segments.index(s) != 0 and segments.index(s) != len(segments)-1:
            if segments[segments.index(s)-1] > segments[segments.index(s)+1]:
                segments[segments.index(s)-1][1] += s[1]
                segments.remove(s)
            else:
                segments[segments.index(s)+1][1] = s[1] + segments[segments.index(s)+1][1]
                segments.remove(s)

    # if neighboring segments are both same type, combine
    for s in segments:
        if segments.index(s) != 0:
            if s[0] == segments[segments.index(s)-1][0]:
                segments[segments.index(s)-1][1] += s[1]
                segments.remove(s)

# calculate minimum speed for each segment
for s in segments:
    min_speed = topspeed
    speed_index = int(np.average(s[1])//1) # average index of points in segment
    for p in s[1]:
        if speed[p] < min_speed:
            min_speed = speed[p]
            speed_index = p
    segments[segments.index(s)].append(min_speed)
    segments[segments.index(s)].append(speed_index)

    if s[0] == 'straight': # mark straights at beginning of segment
        segments[segments.index(s)][5] = s[1][0]

# recalculate distance for each segment
for s in segments:
    segments[segments.index(s)][3] = geopy.distance.distance((gpx.tracks[0].segments[0].points[s[1][0]].latitude, gpx.tracks[0].segments[0].points[s[1][0]].longitude), (gpx.tracks[0].segments[0].points[s[1][-1]].latitude, gpx.tracks[0].segments[0].points[s[1][-1]].longitude)).m

# put speed in mph and turn direction in description of minimum speed point
for s in segments:
    if gpx.tracks[0].segments[0].points[int(s[5])].description == None:
        # mark turns with speed and direction
        if s[0] == 'left' or s[0] == 'right':
            gpx.tracks[0].segments[0].points[int(s[5])].description = '[Corner] ' + s[0] + ' ' + str(math.floor(s[4]*2.23694//10))# + '\n'+ str(np.round(s[2], 2)) + '\n' + str(np.round(s[3], 2)) + '\n' + str(len(s[1]))
        # mark straights with distance
        elif s[0] == 'straight' and s[3] > shortest_straight:
            gpx.tracks[0].segments[0].points[int(s[5])].description = '[Straight] ' + str(np.round(s[3]/1609.34,1)) + ' mi'

# put descriptions into names of points
for p in gpx.tracks[0].segments[0].points:
    if p.description != None:
        p.name = p.description

# create waypoints in track for each point with description
for p in gpx.tracks[0].segments[0].points:
    if p.description != None:
        gpx.waypoints.append(p)

In [4]:
# display map of route with directions
m = folium.Map(location=[gpx.tracks[0].segments[0].points[0].latitude, gpx.tracks[0].segments[0].points[0].longitude], zoom_start=13)

# draw colorline for route with color determined by segment type
for s in segments:
    if s[0] == 'straight':
        folium.PolyLine([[gpx.tracks[0].segments[0].points[p].latitude, gpx.tracks[0].segments[0].points[p].longitude] for p in s[1]], color = 'green', weight=5, opacity=1, popup=gpx.tracks[0].segments[0].points[s[5]].description).add_to(m)
    elif s[0] == 'right':
        folium.PolyLine([[gpx.tracks[0].segments[0].points[p].latitude, gpx.tracks[0].segments[0].points[p].longitude] for p in s[1]], color='blue', weight=5, opacity=1, popup=gpx.tracks[0].segments[0].points[s[5]].description).add_to(m)
    elif s[0] == 'left':
        folium.PolyLine([[gpx.tracks[0].segments[0].points[p].latitude, gpx.tracks[0].segments[0].points[p].longitude] for p in s[1]], color='red', weight=5, opacity=1, popup=gpx.tracks[0].segments[0].points[s[5]].description).add_to(m)

# mark length 1 segments
for s in segments:
    if len(s[1]) == 2:
        folium.Marker([gpx.tracks[0].segments[0].points[s[1][0]].latitude, gpx.tracks[0].segments[0].points[s[1][0]].longitude], popup=s[1], icon=folium.Icon(color='red')).add_to(m)
m


In [5]:
# write new gpx file
with open('processed_test_track.gpx', 'w') as f:
    f.write(gpx.to_xml())